<a href="https://colab.research.google.com/github/jisang93/SNS_Project/blob/master/Insta_Crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [120]:
import requests, re, json, urllib, datetime
import pandas as pd
from bs4 import BeautifulSoup

#search = '%EC%9D%B4%EC%8A%B9%EA%B8%B0'
#def insta(query):
#search = urllib.parse.quote(query)

# 헤더 저장
headers = {
            'cookie' : 'ig_did=1DB997A1-85F5-410B-A44B-DB3A31DE9BF2; mid=XxLtBgALAAHy9DlenU_IjVp8c-vI; shbid=980; shbts=1598087426.6782475; rur=FTW; csrftoken=LGgCJTUqYeDAiBKvjlpy3i6CQ6qlKTd4; ds_user_id=40876776643; sessionid=40876776643%3A9Bs8Y8rng4spJB%3A6; urlgen="{\"211.105.47.207\": 4766\054 \"222.107.238.125\": 4766}:1kAUCE:skN6K55jOiUXtdsowH6nZ-EcpSg"',
            'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36',
            }
    
# 해시 태그 설정
tags = "10대"

res = requests.get('https://www.instagram.com/explore/tags/{}/?__a=1'.format(tags), headers = headers)
res.encoding = 'utf-8'
js = json.loads(res.text)

In [121]:
user_id = []

for i in js['graphql']['hashtag']['edge_hashtag_to_media']['edges']:
    # user 이름으로 접속할 경우 게시글이 다른 게시물로 넘어감
    user_id.append(i['node']['owner']['id'])

user_id = list(set(user_id))
user_id[:5]

['2914108876', '16624086104', '1956213714', '8309799315', '1697883501']

In [122]:
"""
post_user 형태
{"user_ID" : { "user_name", "post_content" : [{"post_code", "post_text", "post_location", "post_time", "post_like"},
                                              {"post_code", "post_text", "post_location", "post_time", "post_like"}]}}}
post_reply
{"post_code" : {'reply_user_name, "reply_content" : [{"reply_text", "reply_time"}]}}                                            
"""

import time

post_users = {}
post_replys = {}
crawl_count = 0
codes = []

for user in user_id:
    id_url = 'https://www.instagram.com/graphql/query/?query_hash=bfa387b2992c3a52dcbe447467b4b771&variables=%7B"id"%3A"{}"%2C"first"%3A12'.format(user)
    url_end = '%7D'
    print(id_url + url_end)
    res1 = requests.get(id_url + url_end, headers=headers)
    res1.encoding = 'utf-8'
    js1 = json.loads(res1.text)

    new_post = {}
    new_reply = {}

    for i in range(len(js1['data']['user']['edge_owner_to_timeline_media']['edges'])):
        # 크롤링 횟수 카운트
        crawl_count += 1
        if crawl_count > 200:
            time.sleep(300)

        # 크롤링 진행 상황 출력
        if int(crawl_count) % 100 == 0:
            print("=" * 50)
            print("크롤링 횟수 : " + str(crawl_count))

        user_id = js1['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['owner']['id'] # 유저 아이디 기준으로 딕셔너리 생성
        post_code = js1['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['shortcode']
        codes.append(post_code)
        print(i, user_id, post_code, crawl_count)

         # 게시물 크롤링

        if user_id not in new_post.keys() :
            new_post[user_id] = {}
        new_post[user_id]['user_name'] = js1['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['owner']['username']
        if js1['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['edge_media_to_caption']['edges'] != []:
            text = js1['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['edge_media_to_caption']['edges'][0]['node']['text']
        else:
            text = None
        if js1['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['location'] != None:
            location = js1['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['location']['name']
        else:
            location = None
        
        if 'post_content' not in new_post[user_id].keys():
            new_post[user_id]['post_content'] = []
        
        new_post[user_id]['post_content'].append({'post_code' : post_code,
                                                  'post_text' : text, 
                                                  'post_time' : (datetime.datetime.fromtimestamp(int(js1['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['taken_at_timestamp']))).strftime('%Y-%m-%d %H:%M:%S'),
                                                  'post_location' : location,
                                                  'post_like' : js1['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['edge_media_preview_like']['count']})
        
        #댓글 크롤링
        if js1['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['edge_media_to_comment']['count'] == 0:
            new_reply[post_code] = None
        else:
            if post_code not in new_reply.keys() :
                new_reply[post_code] = {}

            for j in range(len(js1['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['edge_media_to_comment']['edges'])):
                reply_user_name = js1['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['edge_media_to_comment']['edges'][j]['node']['owner']['username']
                if reply_user_name not in new_reply[post_code].keys():
                    new_reply[post_code][reply_user_name] = []
                new_reply[post_code][reply_user_name].append({'reply_text' : js1['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['edge_media_to_comment']['edges'][j]['node']['text'],
                                                              'reply_time' : (datetime.datetime.fromtimestamp(int(js1['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['edge_media_to_comment']['edges'][j]['node']['created_at']))).strftime('%Y-%m-%d %H:%M:%S')})
        if post_code not in post_replys.keys() :
            post_replys[post_code] = new_reply[post_code]
        else :
            post_replys[post_code]['reply_content'] = post_replys[post_code]['reply_content'] + new_reply[post_code]['reply_content']   
        
    if user_id not in post_users.keys() :
        post_users[user_id] = new_post[user_id]
    else :
        post_users[user_id]['post_content'] = post_users[user_id]['post_content'] + new_post[user_id]['post_content']
    
    print(post_users, post_replys)

https://www.instagram.com/graphql/query/?query_hash=bfa387b2992c3a52dcbe447467b4b771&variables=%7B"id"%3A"2914108876"%2C"first"%3A12%7D
0 2914108876 CETq4RslJaM 1
1 2914108876 CEMv2pklWy1 2
2 2914108876 CEE2ZK0Fsjj 3
3 2914108876 CDT8yi-FN9p 4
4 2914108876 CDEzuW1lnb6 5
5 2914108876 CDEztjQFXWh 6
6 2914108876 CCwFOh9FZ_1 7
7 2914108876 CCTsGn3lwru 8
8 2914108876 CCLxCEblJ_O 9
9 2914108876 CCLw-N6Fsvr 10
10 2914108876 CB5vq4tlVoo 11
11 2914108876 CBtJr80FYbG 12
{'2914108876': {'user_name': 'yss030101', 'post_content': [{'post_code': 'CETq4RslJaM', 'post_text': '스프라이트+샤인머스캣 바\n+마이프로틴 퓨어 카페인 정\n\n#인스타그램 #인스타 #데일리 #수도방위사령부 #소통 #instadaily #today # #팔로워 #경찰특공대 #10대 #헬스 #웨이트 #고2 # #팔로우 #좋아요 #풋살 #축구 #sdt #special_duty_team #udt #under_water_duty_team #체대#태극기 #운동하자', 'post_time': '2020-08-25 09:31:18', 'post_location': None, 'post_like': 9}, {'post_code': 'CEMv2pklWy1', 'post_text': '꽁이사진 투척', 'post_time': '2020-08-22 17:00:05', 'post_location': None, 'post_like': 30}, {'post_code': 'CEE2ZK0Fs

KeyboardInterrupt: ignored

In [123]:
post_replys

{'B6fe7JjHELb': {'__chaeni_._': [{'reply_text': '@hyunehoney 그냥 맛가버림',
    'reply_time': '2019-12-25 17:33:27'}],
  'hyunesnewacc_': [{'reply_text': '@hyunehoney 뻥쟁이 나 배고파 밥조!!!!',
    'reply_time': '2019-12-25 17:33:14'}],
  'iam0gene': [{'reply_text': '@hyunesnewacc_ 내가 더 사랑해 ㅎㅎ 못생긴 사진이 대체어딨어??? 아무리 봐도 이쁜사진 밖에 없어!',
    'reply_time': '2019-12-25 17:32:39'},
   {'reply_text': '@__chaeni_._ 너도 이제 한물 갔냐?',
    'reply_time': '2019-12-25 17:32:59'}]},
 'B7pg5djnSsu': {'0._.2ya': [{'reply_text': '@hyunehoney 이제 아들 낳으러 가즈아🔥',
    'reply_time': '2020-01-23 06:26:29'}],
  'iam0gene': [{'reply_text': '@whyjk_0218 너 친구 아니냐??',
    'reply_time': '2020-01-23 12:28:57'}],
  'song__song_0': [{'reply_text': '이미지가 너무 좋으세요💓 트윈즈 와 함께 활동 하는 beauty model 어떠세요?',
    'reply_time': '2020-03-02 01:52:43'}],
  'whyjk_0218': [{'reply_text': '뭐야 왜 돌하르방이 두개야',
    'reply_time': '2020-01-23 08:30:59'}]},
 'B7sNOdknh5s': {'hyunesnewacc_': [{'reply_text': '감귤공주?!\U0001f9e1\U0001f9e1 쿄쿄 4일동안 알차게 잘먹고 잘 놀다간당 〰️ 버스투어하